<a href="https://colab.research.google.com/github/jyotidabass/Extract-relevant-information-from-text/blob/main/Extract_relevant_information_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

## Load the Natural Language Processing Model

In [2]:
!python -m venv .env
!pip install -U spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/.env/bin/python3

2023-11-27 15:17:53.520936: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 15:17:53.521043: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 15:17:53.521096: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when on

## Preprocessing and Vectorizing

In [11]:
def preprocess_and_vectorize(sentences):
    return np.array([nlp(sent).vector for sent in sentences])

## Sample Data

In [12]:
X = [
    "A study in human volunteers demonstrated that chamomile flavonoids and essential oils penetrate below the skin surface into the deeper skin layers",
    "A study assessed the flavonoid intake of 805 men aged 65–84 years who were followed up for 5 years. Flavonoid intake (analyzed in tertiles) was significantly inversely associated with mortality from coronary heart disease and showed an inverse relation with incidence of myocardial infarction.",
    "In another study, on twelve patients with cardiac disease who underwent cardiac catheterization, hemodynamic measurements obtained prior to and 30 minutes after the oral ingestion of chamomile tea exhibited a small but significant increase in the mean brachial artery pressure",
    "Another non-relevant sentence.",
]
y = [1, 1, 0, 0]  # 1 = relevante, 0 = no relevante

X_vec = preprocess_and_vectorize(X)

## Separate data into Train and Test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

## Use a Model to train (Random Forest, Neural Network, etc)

In [14]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

## Test the Model

In [15]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## Function to use the trained Model

In [16]:
def find_relevant_sentences(text, model, threshold=0.5):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    sent_vecs = preprocess_and_vectorize(sentences)
    probabilities = model.predict_proba(sent_vecs)[:, 1]  # Probabilidades de ser relevante
    relevant_sentences = [sent for sent, prob in zip(sentences, probabilities) if prob >= threshold]
    return relevant_sentences

## Sample Text

In [17]:
text = """
Abstract
Chamomile is one of the most ancient medicinal herbs known to mankind. It is a member of Asteraceae/Compositae family and represented by two common varieties viz. German Chamomile (Chamomilla recutita) and Roman Chamomile (Chamaemelum nobile). The dried flowers of chamomile contain many terpenoids and flavonoids contributing to its medicinal properties. Chamomile preparations are commonly used for many human ailments such as hay fever, inflammation, muscle spasms, menstrual disorders, insomnia, ulcers, wounds, gastrointestinal disorders, rheumatic pain, and hemorrhoids. Essential oils of chamomile are used extensively in cosmetics and aromatherapy. Many different preparations of chamomile have been developed, the most popular of which is in the form of herbal tea consumed more than one million cups per day. In this review we describe the use of chamomile in traditional medicine with regard to evaluating its curative and preventive properties, highlight recent findings for its development as a therapeutic agent promoting human health.

Keywords: chamomile, dietary agents, flavonoids, polyphenols, human health
Go to:
1. INTRODUCTION
The interplay of plants and human health has been documented for thousands of years (1–3). Herbs have been integral to both traditional and non-traditional forms of medicine dating back at least 5000 years (2, 4–6). The enduring popularity of herbal medicines may be explained by the tendency of herbs to work slowly, usually with minimal toxic side effects. One of the most common herbs used for medicinal purposes is chamomile whose standardized tea and herbal extracts are prepared from dried flowers of Matricaria species. Chamomile is one of the oldest, most widely used and well documented medicinal plants in the world and has been recommended for a variety of healing applications (7). Chamomile is a native of the old World and is a member of the daisy family (Asteraceae or Compositae). The hollow, bright gold cones of the flowers are packed with disc or tubular florets and are ringed with about fifteen white ray or ligulate florets, widely represented by two known varieties viz. German chamomile (Matricaria chamomilla) and Roman chamomile (Chamaemelum nobile) (8) . In this review we will discuss the use and possible merits of chamomile, examining its historical use and recent scientific and clinical evaluations of its potential use in the management of various human ailments.

Go to:
2. BIOACTIVE CONSTITUENTS OF CHAMOMILE
Different classes of bioactive constituents are present in chamomile, which have been isolated and used as medicinal preparations and cosmetics (9). The plant contains 0.24%–1.9% volatile oil, composed of a variety of separate oils. When exposed to steam distillation, the oil ranges in color from brilliant blue to deep green when fresh but turns to dark yellow after storage. Despite fading, the oil does not lose its potency. Approximately 120 secondary metabolites have been identified in chamomile, including 28 terpenoids and 36 flavonoids (10, 11). The principal components of the essential oil extracted from the German chamomile flowers are the terpenoids α-bisabolol and its oxide azulenes including chamazulene and acetylene derivatives. Chamazulene and bisabolol are very unstable and are best preserved in an alcoholic tincture. The essential oil of Roman chamomile contains less chamazulene and is mainly constituted from esters of angelic acid and tiglic acid. It also contains farnesene and α-pinene. Roman chamomile contains up to 0.6% of sesquiterpene lactones of the germacranolide type, mainly nobilin and 3-epinobilin. Both α-bisabolol, bisabolol oxides A and B and chamazulene or azulenesse, farnesene and spiro-ether quiterpene lactones, glycosides, hydroxycoumarins, flavanoids (apigenin, luteolin, patuletin, and quercetin), coumarins (herniarin and umbelliferone), terpenoids, and mucilage are considered to be the major bio-active ingredients (12, 13). Other major constituents of the flowers include several phenolic compounds, primarily the flavonoids apigenin, quercetin, patuletin as glucosides and various acetylated derivatives. Among flavonoids, apigenin is the most promising compound. It is present in very small quantities as free apigenin, but predominantly exists in the form of various glycosides (14–18).

Go to:
3. HEALTHCARE PREPRATIONS OF CHAMOMILE
Chamomile is known to be used in various forms of its preparations. Dry powder of chamomile flower is recommended and used by many people for traditionally established health problems. Medicinal ingredients are normally extracted from the dry flowers of chamomile by using water, ethanol or methanol as solvents and corresponding extracts are known as aqueous, ethanolic (alcoholic) and/or methanolic extracts. Optimum chamomile extracts contain about 50 percent alcohol. Normally standardized extracts contain 1.2% of apigenin which is one of the most effective bioactive agents. Aqueous extracts, such as in the form of tea, contain quite low concentrations of free apigenin but include high levels of apigenin-7-O-glucoside. Oral infusion of chamomile is recommended by the German Commission E (19, 20).Chamomile tea is one of the world’s most popular herbal teas and about a million cups are consumed every day. Tea bags of chamomile are also available in the market, containing chamomile flower powder, either pure or blended with other popular medicinal herbs. Chamomile tincture may also be prepared as one part chamomile flower in four parts of water having 12% grain alcohol, which is used to correct summer diarrhea in children and also used with purgatives to prevent cramping. Chamomile flowers are extensively used alone, or combined with crushed poppy-heads, as a poultice or hot foment for inflammatory pain or congestive neuralgia, and in cases of external swelling, such as facial swelling associated with underlying infection or abscess. Chamomile whole plant is used for making herb beers, and also for a lotion, for external application in toothache, earache, neuralgia and in cases of external swelling (20). It is also known to be used as bath additive, recommended for soothing ano-genital inflammation (21). The tea infusion is used as a wash or gargle for inflammation of the mucous membranes of the mouth and throat (22, 23). Inhalation of the vaporized essential oils derived from chamomile flowers is recommended to relieve anxiety, general depression. Chamomile oil is a popular ingredient of aromatherapy and hair care (24, 25). Roman chamomile is widely used in cosmetic preparations and in soothing and softening effect on the skin (26, 27).

Go to:
4. TRADITIONAL USE OF CHAMOMILE
Traditionally, chamomile has been used for centuries as an anti-inflammatory, antioxidant, mild astringent and healing medicine (28). As a traditional medicine, it is used to treat wounds, ulcers, eczema, gout, skin irritations, bruises, burns, canker sores, neuralgia, sciatica, rheumatic pain, hemorrhoids, mastitis and other ailments (29, 30). Externally, chamomile has been used to treat diaper rash, cracked nipples, chicken pox, ear and eye infections, disorders of the eyes including blocked tear ducts, conjunctivitis, nasal inflammation and poison ivy (31, 32). Chamomile is widely used to treat inflammations of the skin and mucous membranes, and for various bacterial infections of the skin, oral cavity and gums, and respiratory tract. Chamomile in the form of an aqueous extract has been frequently used as a mild sedative to calm nerves and reduce anxiety, to treat hysteria, nightmares, insomnia and other sleep problems (33). Chamomile has been valued as a digestive relaxant and has been used to treat various gastrointestinal disturbances including flatulence, indigestion, diarrhea, anorexia, motion sickness, nausea, and vomiting (34, 35). Chamomile has also been used to treat colic, croup, and fevers in children (36). It has been used as an emmenagogue and a uterine tonic in women. It is also effective in arthritis, back pain, bedsores and stomach cramps.

Go to:
5. SCIENTIFIC EVALUATION OF CHAMOMILE
5.1 Anti-inflammatory and antiphlogistic properties
The flowers of chamomile contain 1–2% volatile oils including alpha-bisabolol, alpha-bisabolol oxides A & B, and matricin (usually converted to chamazulene and other flavonoids which possess anti-inflammatory and antiphlogistic properties (12, 19, 35, 36). A study in human volunteers demonstrated that chamomile flavonoids and essential oils penetrate below the skin surface into the deeper skin layers (37). This is important for their use as topical antiphlogistic (anti-inflammatory) agents. One of chamomile’s anti-inflammatory activities involve the inhibition of LPS-induced prostaglandin E(2) release and attenuation of cyclooxygenase (COX-2) enzyme activity without affecting the constitutive form, COX-1 (38).

5.2 Anticancer activity
Most evaluations of tumor growth inhibition by chamomile involve studies with apigenin which is one of the bioactive constituents of chamomile. Studies on preclinical models of skin, prostate, breast and ovarian cancer have shown promising growth inhibitory effects (39–43). In a recently conducted study, chamomile extracts were shown to cause minimal growth inhibitory effects on normal cells, but showed significant reductions in cell viability in various human cancer cell lines. Chamomile exposure induced apoptosis in cancer cells but not in normal cells at similar doses (18). The efficacy of the novel agent TBS-101, a mixture of seven standardized botanical extracts including chamomile has been recently tested. The results confirm it to have a good safety profile with significant anticancer activities against androgen-refractory human prostrate cancer PC-3 cells, both in vitro and in vivo situation (44).

5.3 Common cold
Common cold (acute viral nasopharyngitis) is the most common human disease. It is a mild viral infectious disease of the upper respiratory system. Typically common cold is not life-threatening, although its complications (such as pneumonia) can lead to death, if not properly treated. Studies indicate that inhaling steam with chamomile extract has been helpful in common cold symptoms (45); however, further research is needed to confirm these findings.

5.4 Cardiovascular conditions
It has been suggested that regular use of flavonoids consumed in food may reduce the risk of death from coronary heart disease in elderly men (46). A study assessed the flavonoid intake of 805 men aged 65–84 years who were followed up for 5 years. Flavonoid intake (analyzed in tertiles) was significantly inversely associated with mortality from coronary heart disease and showed an inverse relation with incidence of myocardial infarction. In another study (47), on twelve patients with cardiac disease who underwent cardiac catheterization, hemodynamic measurements obtained prior to and 30 minutes after the oral ingestion of chamomile tea exhibited a small but significant increase in the mean brachial artery pressure. No other significant hemodynamic changes were observed after chamomile consumption. Ten of the twelve patients fell into a deep sleep shortly after drinking the beverage. A large, well-designed randomized controlled trial is needed to assess the potential value of chamomile in improving cardiac health.

5.5 Colic/Diarrhea conditions
An apple pectin-chamomile extract may help shorten the course of diarrhea in children as well as relieve symptoms associated with the condition (47). Two clinical trials have evaluated the efficacy of chamomile for the treatment of colic in children. Chamomile tea was combined with other herbs (German chamomile, vervain, licorice, fennel, balm mint) for administration. In a prospective, randomized, double-blind, placebo-controlled study, 68 healthy term infants who had colic (2 to 8 weeks old) received either herbal tea or placebo (glucose, flavoring). Each infant was offered treatment with every bout of colic, up to 150 mL/dose, no more than three times a day. After 7 days of treatment, parents reported that the tea eliminated the colic in 57% of the infants, whereas placebo was helpful in only 26% (P<0.01). No adverse effects with regard to the number of nighttime awakenings were noted in either group (48). Another study examined the effects of a chamomile extract and apple pectin preparation in 79 children (age 0.5–5.5 y) with acute, non-complicated diarrhea who received either the chamomile/pectin preparation (n = 39) or a placebo (n = 40) for 3 days. Diarrhea ended sooner in children treated with chamomile and pectin (85%), than in the placebo group (58%) (49). These results provide evidence that chamomile can be used safely to treat infant colic disorders.

5.6 Eczema
Topical applications of chamomile have been shown to be moderately effective in the treatment of atopic eczema (50). It was found to be about 60% as effective as 0.25% hydrocortisone cream (51). Roman chamomile of the Manzana type (Kamillosan (R)) may ease discomfort associated with eczema when applied as a cream containing chamomile extract. The Manzana type of chamomile is rich in active ingredients and does not exhibit chamomile-related allergenic potential. In a partially double-blind, randomized study carried out as a half-side comparison, Kamillosan(R) cream was compared with 0.5% hydrocortisone cream and a placebo consisting only of vehicle cream in patients suffering from medium-degree atopic eczema (52). After 2 weeks of treatment, Kamillosan(R) cream showed a slight superiority over 0.5% hydrocortisone and a marginal difference as compared to placebo. Further research is needed to evaluate the usefulness of topical chamomile in managing eczema.

5.7 Gastrointestinal conditions
Chamomile is used traditionally for numerous gastrointestinal conditions, including digestive disorders, "spasm" or colic, upset stomach, flatulence (gas), ulcers, and gastrointestinal irritation (53). Chamomile is especially helpful in dispelling gas, soothing the stomach, and relaxing the muscles that move food through the intestines. The protective effect of a commercial preparation (STW5, Iberogast), containing the extracts of bitter candy tuft, lemon balm leaf, chamomile flower, caraway fruit, peppermint leaf, liquorice root, Angelica root, milk thistle fruit and greater celandine herb, against the development of gastric ulcers has been previously reported (54). STW5 extracts produced a dose dependent anti-ulcerogenic effect associated with a reduced acid output, an increased mucin secretion, an increase in prostaglandin E (2) release and a decrease in leukotrienes. The results obtained demonstrated that STW5 not only lowered gastric acidity as effectively as a commercial antacid, but was more effective in inhibiting secondary hyperacidity (54).

5.8 Hemorrhoids
Studies suggest that chamomile ointment may improve hemorrhoids. Tinctures of chamomile can also be used in a sitz bath format. Tincture of Roman chamomile may reduce inflammation associated with hemorrhoids (55, 56).

5.9 Health Promotion
It has been claimed that consumption of chamomile tea boosts the immune system and helps fight infections associated with colds. The health promoting benefits of chamomile was assessed in a study which involved fourteen volunteers who each drank five cups of the herbal tea daily for two consecutive weeks. Daily urine samples were taken and tested throughout the study, both before and after drinking chamomile tea. Drinking chamomile was associated with a significant increase in urinary levels of hippurate and glycine, which have been associated with increased antibacterial activity (57). In another study, chamomile relieved hypertensive symptoms and decreased the systolic blood pressure significantly, increasing urinary output (58). Additional studies are needed before a more definitive link between chamomile and its alleged health benefits can be established.

5.10 Inflammatory conditions
Inflammation is associated with many gastrointestinal disorders complaints, such as esophageal reflux, diverticular disease, and inflammatory disease (59–61). Studies in preclinical models suggest that chamomile inhibits Helicobacter pylori, the bacteria that can contribute to stomach ulcers (60). Chamomile is believed to be helpful in reducing smooth muscle spasms associated with various gastrointestinal inflammatory disorders. Chamomile is often used to treat mild skin irritations, including sunburn, rashes, sores and even eye inflammations (62–65) but its value in treating these conditions has not been shown with evidence-based research.

5.11 Mucositis
Mouth ulcers are a common condition with a variety of etiologies (66). Stomatitis is a major dose-limiting toxicity from bolus 5-fluorouracil-based (5-FU) chemotherapy regimens. A double-blind, placebo-controlled clinical trial including 164 patients was conducted (22). Patients were entered into the study at the time of their first cycle of 5-FU-based chemotherapy and were randomized to receive a chamomile or placebo mouthwash thrice daily for 14 days. There was no suggestion of any stomatitis difference between patients randomized to either protocol arm. There was also no suggestion of toxicity. Similar results were obtained with another prospective trial on chamomile in this situation. Data obtained from these clinical trials did not support the pre study hypothesis that chamomile could decrease 5-FU-induced stomatitis. The results remain unclear if chamomile is helpful in this situation.

5.12 Osteoporosis
Osteoporosis is a metabolic bone disease resulting from low bone mass (osteopenia) due to excessive bone resorption. Sufferers are prone to bone fractures from relatively minor trauma. Agents which include selective estrogen receptor modulators or SERMs, biphosphonates, calcitonin are frequently used to prevent bone loss. To prevent bone loss that occurs with increasing age, chamomile extract was evaluated for its ability to stimulate the differentiation and mineralization of osteoblastic cells. Chamomile extract was shown to stimulate osteoblastic cell differentiation and to exhibit an anti-estrogenic effect, suggesting an estrogen receptor-related mechanism (67). However, further studies are needed before it can be considered for clinical use.

5.13 Sleep aid/sedation
Traditionally, chamomile preparations such as tea and essential oil aromatherapy have been used to treat insomnia and to induce sedation (calming effects). Chamomile is widely regarded as a mild tranquillizer and sleep-inducer. Sedative effects may be due to the flavonoid, apigenin that binds to benzodiazepine receptors in the brain (68). Studies in preclinical models have shown anticonvulsant and CNS depressant effects respectively. Clinical trials are notable for their absence, although ten cardiac patients are reported to have immediately fallen into a deep sleep lasting for 90 minutes after drinking chamomile tea (47). Chamomile extracts exhibit benzodiazepine-like hypnotic activity (69). In another study, inhalation of the vapor of chamomile oil reduced a stress-induced increase in plasma adrenocorticotropic hormone (ACTH) levels. Diazepam, co-administered with the chamomile oil vapor, further reduced ACTH levels, while flumazenile, a BDZ antagonist blocked the effect of chamomile oil vapor on ACTH. According to Paladini et al. (70), the separation index (ratio between the maximal anxiolytic dose and the minimal sedative dose) for diazepam is 3 while for apigenin it is 10. Compounds, other than apigenin, present in extracts of chamomile can also bind BDZ and GABA receptors in the brain and might be responsible for some sedative effect; however, many of these compounds are as yet unidentified.

5.14 Anxiety and seizure
Chamomile has been reported in the treatment of generalized anxiety disorder (GAD). But the reports seem contradictory as an earlier report suggests that German chamomile showed significant inhibition of GAD activity (71). The recent results from the controlled clinical trial on chamomile extract for GAD suggests that it may have modest anxiolytic activity in patients with mild to moderate GAD (72). Extracts of chamomile (M. recutita) possess suitable effects on seizure induced by picrotoxin (73). Furthermore, apigenin has been shown to reduce the latency in the onset of picrotoxin-induced convulsions and reduction in locomotor activity but did not demonstrate any anxiolytic, myorelaxant, or anticonvulsant activities (16).

5.15 Diabetes
Studies suggest that chamomile ameliorates hyperglycemia and diabetic complications by suppressing blood sugar levels, increasing liver glycogen storage and inhibition of sorbitol in the human erythrocytes (74). The pharmacological activity of chamomile extract has shown to be independent of insulin secretion (75), and studies further reveal its protective effect on pancreatic beta cells in diminishing hyperglycemia-related oxidative stress (76). Additional studies are required to evaluate the usefulness of chamomile in managing diabetes.

5.16 Sore throat/hoarseness
The efficacy of lubrication of the endo-tracheal tube cuff with chamomile before intubation on postoperative sore throat and hoarseness was determined in a randomized double-blind study. 161 patients whose American Society of Anesthesiologists (ASA) physical status was I or II, and undergoing elective surgical, orthopedic, gynecological or urological surgeries were divided in two groups. The study group received 10 puffs of chamomile extract (Kamillosan M spray, total 370 mg of Chamomile extract) at the site of the cuff of the endotracheal tube for lubrication, while the control group did not receive any lubrication before intubations. Standard general anesthesia with tracheal intubations was given in both groups. 41 out of 81 patients (50.6%) in the chamomile group reported no postoperative sore throat in the post-anesthesia care unit compared with 45 out of 80 patients (56.3%) in the control group. Postoperative sore throat and hoarseness both in the post-anesthesia care unit and at 24 h post-operation were not statistically different. Lubrication of endo-tracheal tube cuff with chamomile extract spray before intubations can not prevent post operative sore throat and hoarseness (77). Similar results were obtained in another double blind study (78).

5.17 Vaginitis
Vaginal inflammation is common in women of all ages. Vaginitis is associated with itching, vaginal discharge, or pain with urination. Atrophic vaginitis most commonly occurs in menopausal and postmenopausal women, and its occurrence is often associated with reduced levels of estrogen. Chamomile douche may improve symptoms of vaginitis with few side effects (79). There is insufficient research data to allow conclusions concerning possible potential benefits of chamomile for this condition.

5.18 Wound healing
The efficacy of topical use of chamomile to enhance wound healing was evaluated in a double-blind trial on 14 patients who underwent dermabrasion of tattoos. The effects on drying and epithelialization were observed, and chamomile was judged to be statistically efficacious in producing wound drying and in speeding epithelialization (80). Antimicrobial activity of the extract against various microorganisms was also assessed. The test group, on day 15, exhibited a greater reduction in the wound area when compared with the controls (61 % versus 48%), faster epithelialization and a significantly higher wound-breaking strength. In addition, wet and dry granulation tissue weight and hydroxyproline content were significantly higher. The increased rate of wound contraction, together with the increased wound-breaking strength, hydroxyproline content and histological observations, support the use of M. recutita in wound management (81). Recent studies suggest that chamomile caused complete wound healing faster than corticosteroids (82). However, further studies are needed before it can be considered for clinical use.

5.19 Quality-of-life in cancer patients
Essential oils obtained from Roman chamomile are the basic ingredients of aromatherapy. Clinical trials of aromatherapy in cancer patients have shown no statistically significant differences between treated and untreated patients (83). Another pilot study investigated the effects of aromatherapy massage on the anxiety and self-esteem experience in Korean elderly women. A quasi-experimental, control group, pretest-posttest design used 36 elderly females: 16 in the experimental group and 20 in the control group. Aromatherapy massage using lavender, chamomile, rosemary, and lemon was given to the experimental group only. Each massage session lasted 20 min, and was performed 3 times per week for two 3-week periods with an intervening 1-week break. The intervention produced significant differences in the anxiety and self-esteem. These results suggest that aromatherapy massage exerts positive effects on anxiety and self-esteem (84–86). However, more objective, clinical measures should be applied in a future study with a randomized placebo-controlled design.

Go to:
6.0 CONTRAINDICATIONS AND SAFETY ISSUES WITH CHAMOMILE
A relatively low percentage of people are sensitive to chamomile and develop allergic reactions (87). People sensitive to ragweed and chrysanthemums or other members of the Compositae family are more prone to develop contact allergies to chamomile, especially if they take other drugs that help to trigger the sensitization. A large-scale clinical trial was conducted in Hamburg, Germany, between 1985 and 1991 to study the development of contact dermatitis secondary to exposure to a mixture of components derived from the Compositae family. Twelve species of the Compositae family, including German chamomile, were selected and tested individually when the mixture induced allergic reactions. During the study, 3,851 individuals were tested using a patch with the plant extract (88). Of these patients, 118 (3.1%) experienced an allergic reaction. Further tests revealed that feverfew elicited the most allergic reactions (70.1% of patients) followed by chrysanthemums (63.6%) and tansy (60.8%). Chamomile fell in the middle range (56.5%). A study involving 686 subjects exposed either to a sesquiterpene lactone mixture or a mixture of Compositae extracts led to allergic reactions in 4.5% of subjects (89). In another study it was shown that eye washing with chamomile tea in hay fever patients who have conjunctivitis exacerbates the eye inflammation, whereas no worsening of eye inflammation was noted when chamomile tea was ingested orally (90). Chamomile is listed on the FDA's GRAS (generally recognized as safe) list. It is possible that some reports of allergic reactions to chamomile may be due to contamination of chamomile by "dog chamomile," a highly allergenic and bad-tasting plant of similar appearance. Evidence of cross-reactivity of chamomile with other drugs is not well documented, and further study of this issue is needed prior to reaching conclusions. Safety in young children, pregnant or nursing women, or those with liver or kidney disease has not been established, although there have not been any credible reports of toxicity caused by this common beverage tea.

Go to:
7.0 CONCLUSIONS
Chamomile has been used as an herbal medication since ancient times, is still popular today and probably will continue to be used in the future because it contains various bioactive phytochemicals that could provide therapeutic effects. Chamomile can help in improving cardiovascular conditions, stimulate immune system and provide some protection against cancer. Establishing whether or not therapeutic effects of chamomile are beneficial to patients will require research and generation of scientific evidence. There is a need for continued efforts that focuses on pre-clinical studies with chamomile involving animal models of various diseases. This may then be consequently validated in clinical trials that will help in developing chamomile as a promising therapeutic agent. Without such evidence, it will remain unclear whether these untested and unproven medical treatments are truly beneficial. It is advisable that the discriminate and proper use of chamomile preparations could be safe and provide therapeutic benefits however the indiscriminate or improper use can be unsafe and harmful.
"""

## Extract relevant info from text

In [18]:
relevant_sentences = find_relevant_sentences(text, clf, threshold=0.5)
print(len(relevant_sentences))
for sent in relevant_sentences:
    print(f"•\t{sent}")

1
•	A study in human volunteers demonstrated that chamomile flavonoids and essential oils penetrate below the skin surface into the deeper skin layers (37).
